In [1]:
import pandas as pd 
import numpy as np 
import os
from unidecode import unidecode
from datetime import datetime, date, timedelta
import xlrd
from xlrd.xldate import xldate_as_tuple
import ConeccionesSucursales as cs
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta = '/home/estadistico/Documents/Erick/'
mes = 20210101
mes2 = 'Enero2021'
mes3 = pd.to_datetime('01/01/2021',dayfirst=True)
fin_mes = pd.to_datetime('31/01/2021',dayfirst=True)
quetzal = 2.5
dolar = 20
meta_baz_renta = 1004000
day=26

# Lectura Pagos

In [3]:
#Lectura pagos Bradesco
book = os.listdir(''+ruta+'/'+str(mes2)+'/Bradesco/Pagos/')
pagos_brad = pd.DataFrame()
for i in book:
    pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Bradesco/Pagos/'+i+'')
    pagos_new = pagos_new.iloc[:,0:9]
    pagos_brad = pd.concat([pagos_brad,pagos_new],ignore_index=True)
    pagos_brad = pagos_brad.drop_duplicates()
pagos_brad = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad = pd.DataFrame(pagos_brad.to_records())
pagos_brad = pagos_brad.loc[pagos_brad['FECHA_PAGO']>=mes3]

In [4]:
#Lectura pagos Liverpool
book =  os.listdir(''+ruta+'/'+str(mes2)+'/Liverpool/Pagos/')
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
pagos_liv_new = pd.DataFrame()
for i in book:
    for j in sheet_liv:
        pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Liverpool/Pagos/'+i+'',sheet_name = str(j),converters={'Cuenta':str})
        pagos_new.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
        pagos_liv_new = pd.concat([pagos_liv_new,pagos_new],ignore_index=True)
        pagos_liv_new = pagos_liv_new.iloc[:,0:6]
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
    pagos_liv = pagos_liv.drop_duplicates()
pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
pagos_liv = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv = pd.DataFrame(pagos_liv.to_records())
pagos_liv = pagos_liv.loc[pagos_liv['Fecha Pago']>=mes3]

In [5]:
#Lectura pagos Credifiel
book = os.listdir(''+ruta+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for j in book:
    pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Credifiel/Pagos/'+j+'')
    pagos_cred = pd.concat([pagos_cred,pagos_new],ignore_index=True)
    pagos_cred = pagos_cred.drop_duplicates()
#pagos_cred = pagos_cred.iloc[:,[12,1,8]]
pagos_cred = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred = pd.DataFrame(pagos_cred.to_records())
pagos_cred = pagos_cred.loc[pagos_cred['FECHA']>=mes3]

In [6]:
#Lectura pagos Banco Azteca Renta
book = os.listdir(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta/')
pagos_baz_renta = pd.DataFrame()
for i in book:
    pagos_new = pd.read_csv(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta/'+i+'')
    pagos_baz_renta = pd.concat([pagos_baz_renta,pagos_new],ignore_index=True)
pagos_baz_renta = pagos_baz_renta.drop_duplicates()
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FNMONTOPAGADO']>0]
pagos_baz_renta['FDFECHAABONO'] = pd.to_datetime(pagos_baz_renta['FDFECHAABONO'],dayfirst=True)
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FDFECHAABONO']>=mes3]
pagos_baz_renta = pd.pivot_table(pagos_baz_renta,index=['CAMPANAID','FDFECHAABONO','FIIDPERIODO','FNSCOMPROMISO','FCEMPNUMCORTE','FNMONTOPROMETIDO'],values=['FNMONTOPAGADO'],aggfunc=np.sum)
pagos_baz_renta = pd.DataFrame(pagos_baz_renta.to_records())
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FDFECHAABONO']>=mes3]
pagos_baz_renta['Cliente'] = 'Banco Azteca'
pagos_baz_renta['Division'] = 'Renta'

In [7]:
#Lectura pagos Banco Azteca Recovery
book = os.listdir(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/')
pagos_baz_recovery = pd.DataFrame()
for i in book:
    pagos_new = pd.read_csv(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/'+i+'',sep='|')
    pagos_baz_recovery = pd.concat([pagos_baz_recovery,pagos_new],ignore_index=True)
pagos_baz_recovery['DIA'] = pd.to_datetime(pagos_baz_recovery['DIA'],dayfirst=True)
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery['credito'] = pagos_baz_recovery['PAIS'].map(str)+'-'+pagos_baz_recovery['CANAL'].map(str)+'-'+pagos_baz_recovery['SUCURSAL'].map(str)+'-'+pagos_baz_recovery['FOLIO'].map(str)
pagos_baz_recovery = pagos_baz_recovery.drop_duplicates()
pagos_baz_recovery['Cliente'] = 'Banco Azteca'
pagos_baz_recovery['Division'] = 'Recovery'

In [8]:
#Lectura pagos Banco Azteca 26-39
book = os.listdir(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/')
pagos_baz_26 = pd.DataFrame()
for i in book:
    pagos_new = pd.read_csv(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/'+i+'',sep='|')
    pagos_baz_26 = pd.concat([pagos_baz_26,pagos_new],ignore_index=True)
pagos_baz_26['DIA'] = pd.to_datetime(pagos_baz_26['DIA'],dayfirst=True)
pagos_baz_26 = pagos_baz_26.loc[pagos_baz_26['DIA']>=mes3]
pagos_baz_26['credito'] = pagos_baz_26['PAIS'].map(str)+'-'+pagos_baz_26['CANAL'].map(str)+'-'+pagos_baz_26['SUCURSAL'].map(str)+'-'+pagos_baz_26['FOLIO'].map(str)
pagos_baz_26 = pagos_baz_26.drop_duplicates()
pagos_baz_26['Cliente'] = 'Banco Azteca'
pagos_baz_26['Division'] = '26-39'

In [9]:
#Leemos facturacion Banco Azteca recovery
book = os.listdir(''+ruta+'/'+str(mes2)+'/Banco Azteca/Facturacion/Recovery/')
fact_baz_rec = pd.DataFrame()
for i in book:
    pagos_new = pd.read_csv(''+ruta+'/'+str(mes2)+'/Banco Azteca/Facturacion/Recovery/'+i+'',encoding='utf-8')
    fact_baz_rec = pd.concat([fact_baz_rec,pagos_new],ignore_index=True)
fact_baz_rec = fact_baz_rec.drop_duplicates()
fact_baz_rec['Final de Semana'] = pd.to_datetime(fact_baz_rec['Final de Semana'],dayfirst=True)
fact_baz_rec['Inicio de Semana'] = pd.to_datetime(fact_baz_rec['Inicio de Semana'],dayfirst=True)
fact_baz_rec['Division'] = 'Recovery'
fact_baz_rec['Cliente'] = 'Banco Azteca'

In [10]:
#Leemos facturacion Banco Azteca recovery
book = os.listdir(''+ruta+'/'+str(mes2)+'/Banco Azteca/Facturacion/26_39/')
fact_baz_26_55 = pd.DataFrame()
for i in book:
    pagos_new = pd.read_csv(''+ruta+'/'+str(mes2)+'/Banco Azteca/Facturacion/26_39/'+i+'',encoding='utf-8')
    fact_baz_26_55 = pd.concat([fact_baz_26_55,pagos_new],ignore_index=True)
fact_baz_26_55 = fact_baz_26_55.drop_duplicates()
fact_baz_26_55['Final de Semana'] = pd.to_datetime(fact_baz_26_55['Final de Semana'],dayfirst=True)
fact_baz_26_55['Inicio de Semana'] = pd.to_datetime(fact_baz_26_55['Inicio de Semana'],dayfirst=True)
fact_baz_26_55['Division'] = '26_39'
fact_baz_26_55['Cliente'] = 'Banco Azteca'

In [11]:
#Lectura pagos Oriflam
book = os.listdir(''+ruta+'/'+mes2+'/Torreon/Oriflame/Pagos/')
pagos_oriflame = pd.DataFrame()
for i in book:
    pagos_new = pd.read_excel(''+ruta+'/'+mes2+'/Torreon/Oriflame/Pagos/'+i+'')
    pagos_oriflame = pd.concat([pagos_oriflame,pagos_new],ignore_index=True)
    pagos_oriflame = pagos_oriflame.drop_duplicates()
pagos_oriflame['Cliente'] = 'Oriflame'
pagos_oriflame['Factura'] = pagos_oriflame['PAGO_TOTAL']*.1
pagos_oriflame = pagos_oriflame.loc[pagos_oriflame['FECHA_PAGO']>=mes3]

In [12]:
#Lectura pagos Arabela
book = os.listdir(''+ruta+'/'+mes2+'/Torreon/Arabela/Pagos/')
pagos_arabela = pd.DataFrame()
for i in book:
    pagos_new = pd.read_excel(''+ruta+'/'+mes2+'/Torreon/Arabela/Pagos/'+i+'')
    pagos_arabela = pd.concat([pagos_arabela,pagos_new],ignore_index=True)
    pagos_arabela = pagos_arabela.drop_duplicates()
pagos_arabela['Cliente'] = 'Arabela'
pagos_arabela['FechaEntrega'] = [datetime.strptime(str(x), '%Y%m%d').strftime('%d/%m/%Y') for x in pagos_arabela['FechaEntrega']]
pagos_arabela['FechaEntrega'] = pd.to_datetime(pagos_arabela['FechaEntrega'],dayfirst=True)
pagos_arabela = pagos_arabela.loc[pagos_arabela['FechaEntrega']>=mes3]
pagos_arabela['factura'] = pagos_arabela['pago']*.22

In [13]:
#Lectura pagos invex
book = os.listdir(''+ruta+''+mes2+'/Torreon/Invex/Pagos/')
pagos_invex = pd.DataFrame()
for i in book:
    pagos_new = pd.read_excel(''+ruta+'/'+mes2+'/Torreon/Invex/Pagos/'+i+'')
    pagos_invex = pd.concat([pagos_invex,pagos_new],ignore_index=True)
pagos_invex['Cliente'] = 'Invex_'+pagos_invex['Agencia'].map(str)
pagos_invex = pagos_invex.drop_duplicates()
# pagos_invex = pagos_invex.loc[pagos_invex['TRAN_POST_DT']>=mes3]

# Lectura asignaciones

In [14]:
#Asignacion Liverpool
asignacion_liv = pd.read_csv(''+ruta+'/Liverpool/Asignacion/Asignaciones csv/Asignacion '+mes2+'.csv',converters={'# CUENTA':str})
asignacion_liv = asignacion_liv[['ID AGENCIA', '# CUENTA']]
asignacion_liv.columns = ['division','credito']

In [15]:
#Asignacion Bradesco
asignacion_brad_c0 = pd.read_csv(''+ruta+'/Bradesco/Asignacion_c0/CSV/ASIGNACION C0 '+mes2+'.csv')
asignacion_brad_c0 = asignacion_brad_c0.iloc[:,[0,2,3,8,10,11,14,25]]
asignacion_brad_extra = pd.read_csv(''+ruta+'/Bradesco/Asignacion_extra/CSV/Asignacion Extra '+mes2+'.csv',encoding= 'unicode_escape')
asignacion_brad_extra = asignacion_brad_extra.iloc[:,[0,2,3,5,9,25,26,28]]
asignacion_brad_extra.columns = ['CODIGO_ORG','NUMERO_CUENTA','NUMERO_TARJETA','RFC','ESTADO','SALDO_TOTAL','PAGO_DESCUENTO','PAGOSVENCIDOS']

asignacion_brad = pd.concat([asignacion_brad_c0,asignacion_brad_extra],ignore_index=True)

In [16]:
#Hacemos la consulta de la asignacion
book = os.listdir(''+ruta+'/'+mes2+'/Credifiel/Asignacion/')
asignacion_cred = pd.DataFrame()
for i in book:
    new = pd.read_excel(''+ruta+'/Credifiel/Asignacion/Asignacion Credifiel '+mes2+'.xlsx')
    asignacion_cred = pd.concat([asignacion_cred,new],ignore_index=True)
    asignacion_cred = asignacion_cred[['CREDITO','ARANCEL']]
    asignacion_cred = asignacion_cred.drop_duplicates(subset=['CREDITO'])
asignacion_cred['CREDITO'] = pd.to_numeric(asignacion_cred['CREDITO'])

In [17]:
# #Asignacion Banco Azteca Renta
# book = os.listdir(''+ruta+'/'+mes2+'/Banco Azteca/Asignacion/Renta/')
# asignacion_renta = pd.DataFrame()
# for i in book:
#     asignacion_new = pd.read_excel(''+ruta+'/'+mes2+'/Banco Azteca/Asignacion/Renta/'+i+'')
#     asignacion_renta = pd.concat([asignacion_renta,asignacion_new],ignore_index=True)

# Parametros a ajustar

In [18]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 1204704, .07
c04pv2, arancel_4pv2 = 1807056, .09
c04pv3, arancel_4pv3 = 2409407, .12
c05pv1, arancel_5pv1 = 662269, .09
c05pv2, arancel_5pv2 = 993403, .10
c05pv3, arancel_5pv3 = 1324538, .14
c06pv1, arancel_6pv1 = 807172, .10
c06pv2, arancel_6pv2 = 1210759, .11
c06pv3, arancel_6pv3 = 1614345, .16
a1, arancel_a1 = 260190, .27
a2, arancel_a2 = 390285, .30
a3, arancel_a3 = 520381, .33
b1, arancel_b1 = 215083, .25
b2, arancel_b2 = 322624, .28
b3, arancel_b3 = 430165, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 626634, .19
c22, arancel_c22 = 939951, .22
c23, arancel_c23 = 1253267, .25
c31, arancel_c31 = 246425, .21
c32, arancel_c32 = 369637, .24
c33, arancel_c33 = 492849, .27
c41, arancel_c41 = 123529, .23
c42, arancel_c42 = 185294, .26
c43, arancel_c43 = 247059, .29

In [19]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
cof, arancel_cof,arancel_cof2 = 300000, .5, .5
sub, arancel_sub, arancel_sub2 = 300000, .15, .2
subdic, arancel_subdic, arancel_subdic2 = 350000, .16, .2
n2, arancel_n2, arancel_n22 = 650000, .4, .5
pren, arancel_pren, arancel_pren2 = 300000, .28, .28

In [20]:
#Metas Naucalpan
meta_naucalpan = pd.read_excel(''+ruta+'Metas/Metas '+mes2+'.xlsx',sheet_name='Naucalpan')
#Metas Torreon
meta_torreon = pd.read_excel(''+ruta+'Metas/Metas '+mes2+'.xlsx',sheet_name='Torreon')
meta_torreon

,Cliente,Meta,Facturacion,FechaFact
0,Avon,890000,171972.03,2021-01-20
1,Oriflame,65000,38500.00,2021-01-26
2,Invex,1095000,819614.00,2021-01-26
3,Arabela,95000,65106.00,2021-01-26
4,Cimaco,3500,547.00,2021-01-10
5,Cristian Lay,6000,2896.00,2021-01-24


# Analisis

In [21]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H-%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [22]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

# $\textbf{Naucalpan}$

# Liverpool

In [23]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv_mes = cs.query_naucalpan('liverpool_rpt_gestiones_detallado',mes,today)
gestion_liv_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv_mes = gestion_liv_mes.loc[gestion_liv_mes['dictamen']=='PROMESA']
gestion_liv_mes['anio'] = [int(x[:4]) for x in gestion_liv_mes['fecha_promesa']]
gestion_liv_mes = gestion_liv_mes.loc[(gestion_liv_mes['anio']>2010)&(gestion_liv_mes['anio']<2100)]
gestion_liv_mes['fecha_promesa'] = pd.to_datetime(gestion_liv_mes['fecha_promesa'],dayfirst=False)

In [24]:
#Hacemos el consolidado
consolidado_liv_mes = pd.merge(gestion_liv_mes,asignacion_liv,how='left',on=['credito'])
cols = ['monto_promesa']
consolidado_liv_mes[cols] = consolidado_liv_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']>50)]
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']<500000)]
consolidado_liv_mes['division'].fillna('COBRANZA',inplace=True)
consolidado_liv_mes['division'] = consolidado_liv_mes['division'].replace('','COBRANZA')
consolidado_liv_mes = consolidado_liv_mes.drop_duplicates(subset = ['usuario','credito','fecha_promesa','monto_promesa'])

In [25]:
productos = [('DKN00REM','Liverpool','Penacional'),('DKN02REM','Liverpool Nacional','Nacional2'),('DKN04REM','Liverpool Cof','Cof'),('SA001REM','Liverpool','Suburbia'),('SA002REM','Liverpool','Dictamen')]
productos = pd.DataFrame(productos)
productos.columns = ['division','id','segmento']
consolidado_liv_mes = pd.merge(consolidado_liv_mes,productos,how='left',on=['division'])

In [26]:
#Obtenemos los pagos totales por division
pagos_totales_liv = pd.pivot_table(pagos_liv,index=['Id'],values=['Monto'],aggfunc=[np.sum])
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
#pagos_totales_liv = pagos_totales_liv2.iloc[:,[0,2,1]]
pagos_totales_liv.columns = ['Division','SumaPagos']
pagos_totales_liv.fillna(0,inplace=True)

In [27]:
#Para cada segmento se le asigna lo que lleva de cada uno
DKN00REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN00REM', 'SumaPagos'].item()
DKN02REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN02REM', 'SumaPagos'].item()
DKN04REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN04REM', 'SumaPagos'].item()
SA001REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='SA001REM', 'SumaPagos'].item()
SA002REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='SA002REM', 'SumaPagos'].item()

In [28]:
#Funcion que asignará el arancel correspondiente
def arancel_liv(row):
    if row['Id'] == 'DKN00REM' :
        return np.where((DKN00REM>0) & (DKN00REM<=pren/arancel_pren),arancel_pren,arancel_pren2)
    elif row['Id'] == 'DKN02REM':
        return np.where((DKN02REM>0) & (DKN02REM<=n2/arancel_n2),arancel_n2,arancel_n22)
    elif row['Id'] == 'DKN04REM':
        return np.where((DKN04REM>0) & (DKN04REM<=cof/arancel_cof),arancel_cof,arancel_cof2)
    elif row['Id'] == 'SA001REM':
        return np.where((SA001REM>0) & (SA001REM<=sub/arancel_sub),arancel_sub,arancel_sub2)
    elif row['Id'] == 'SA002REM':
        return np.where((SA002REM>0) & (SA002REM<=subdic/arancel_subdic),arancel_subdic,arancel_subdic2)
    else:
        return 0

In [29]:
#Aplicamos la funcion para obtener los aranceles
pagos_liv2 = pd.merge(pagos_liv,productos,how='left',left_on='Id',right_on='division')
pagos_liv2['Arancel'] = pagos_liv2.apply(arancel_liv, axis=1)

#Agregamos aranceles al consolidado
pagos_liv2['PagoArancel'] = [x*y for x,y in zip(pagos_liv2['Monto'],pagos_liv2['Arancel'])]
pagos_liv2['Fecha Pago'] = pd.to_datetime(pagos_liv2['Fecha Pago'],dayfirst=True)
pagos_liv2.head(1)

,Id,Cuenta,Fecha Pago,Fecha Prom,Monto,division,id,segmento,Arancel,PagoArancel
0,DKN00REM,0000013000000133855,2021-01-08,07/01/2021,4797.0,DKN00REM,Liverpool,Penacional,0.28,1343.16


In [30]:
#Total pagos Liver registrados
pagos_liv2['Cliente'] = 'Liverpool'
pagos_totales_liv2 = pd.pivot_table(pagos_liv2,index=['Cliente'],values=['PagoArancel'],aggfunc=np.sum,dropna=False)
pagos_totales_liv2 = pd.DataFrame(pagos_totales_liv2.to_records())
#pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv2.columns = ['Cliente','Facturacion']
pagos_totales_liv2.fillna(0,inplace=True)
pagos_totales_liv2['Facturacion'] = pagos_totales_liv2['Facturacion'].round(2)
pagos_totales_liv2['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_totales_liv2['FechaFact'] = pd.to_datetime(pagos_totales_liv2['FechaFact'],dayfirst=True)
pagos_totales_liv2

,Cliente,Facturacion,FechaFact
0,Liverpool,651794.98,2021-01-25


In [31]:
#Funcion que asigna el siguiente arancel
def sig_arancel_liv(row):
    if row['Asignacion'] == 'DKN00REM' :
        return np.where((DKN00REM>0) & (DKN00REM<=pren/arancel_pren),round(pren/arancel_pren,1),0)
    elif row['Asignacion'] == 'DKN02REM':
        return np.where((DKN02REM>0) & (DKN02REM<=n2/arancel_n2),n2/arancel_n2,0)
    elif row['Asignacion'] == 'DKN04REM':
        return np.where((DKN04REM>0) & (DKN04REM<=cof/arancel_cof),cof/arancel_cof,0)
    elif row['Asignacion'] == 'SA001REM':
        return np.where((SA001REM>0) & (SA001REM<=sub/arancel_sub),sub/arancel_sub,0)
    elif row['Asignacion'] == 'SA002REM':
        return np.where((SA002REM>0) & (SA002REM<=subdic/arancel_subdic),subdic/arancel_subdic,0)
    else:
        return 0

In [32]:
#Aplicamos 
pagos_liv_gerencia = pd.pivot_table(pagos_liv2,index=['Id'],values=['Monto'],aggfunc=[np.sum])
pagos_liv_gerencia = pd.DataFrame(pagos_liv_gerencia.to_records())
pagos_liv_gerencia.columns = ['Asignacion','Recuperado']
pagos_liv_gerencia['SiguienteArancel'] = pagos_liv_gerencia.apply(sig_arancel_liv, axis=1)
pagos_liv_gerencia['RestSigArancel'] = pagos_liv_gerencia['SiguienteArancel']-pagos_liv_gerencia['Recuperado']
cols = ['Recuperado','SiguienteArancel','RestSigArancel']
pagos_liv_gerencia[cols] = pagos_liv_gerencia[cols].round(2)
pagos_liv_gerencia['RestSigArancel'] = np.where(pagos_liv_gerencia['RestSigArancel']<0,'Alcanzado',pagos_liv_gerencia['RestSigArancel'])
pagos_liv_gerencia = pd.merge(pagos_liv_gerencia,productos,how='left',left_on='Asignacion',right_on='division')
pagos_liv_gerencia = pagos_liv_gerencia.iloc[:,[6,1,2,3]]
pagos_liv_gerencia.columns = ['Division','Recuperado','SiguienteArancel','RestSigArancel']

In [33]:
#Unimos el consolidado con los pagos 
consolidado_pagos_liv = pd.merge(consolidado_liv_mes,pagos_liv2[['Id','Cuenta','Fecha Pago','Monto','id','segmento','Arancel','PagoArancel']],how='left',left_on=['credito'],right_on=['Cuenta'])
consolidado_pagos_liv = consolidado_pagos_liv.drop_duplicates(subset = ['credito','Monto','usuario','fecha_promesa'])
consolidado_pagos_liv['Fecha Pago'] = pd.to_datetime(consolidado_pagos_liv['Fecha Pago'],dayfirst=True)
consolidado_pagos_liv['dia_prom'] = consolidado_pagos_liv['fecha_promesa'].dt.day
consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['Fecha Pago'].dt.day
cols = ['dia_prom','dia_pago','Monto']
consolidado_pagos_liv[cols] = consolidado_pagos_liv[cols].apply(pd.to_numeric, errors='coerce',downcast='integer')
consolidado_pagos_liv['dia_pago'].fillna(0,inplace=True)
consolidado_pagos_liv['dia_pago'] = [int(x) for x in consolidado_pagos_liv['dia_pago']]

In [34]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consolidado_pagos_liv['PagoReal'] = [np.where((z < x+pd.Timedelta(3, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consolidado_pagos_liv['fecha_promesa'],consolidado_pagos_liv['Monto'],consolidado_pagos_liv['Fecha Pago'])]
consolidado_pagos_liv['PagoReal'] = [x.item() for x in consolidado_pagos_liv['PagoReal']]
consolidado_pagos_liv['MontoArancel'] = [x*y for x,y in zip(consolidado_pagos_liv['PagoReal'],consolidado_pagos_liv['Arancel'])]

In [35]:
#Total pagos Liver registrados
pagos_totales_liv3 = pd.pivot_table(pagos_liv2,index=['segmento'],values=['PagoArancel'],aggfunc=np.sum,dropna=False)
pagos_totales_liv3 = pd.DataFrame(pagos_totales_liv3.to_records())
#pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv3.columns = ['Division','Facturacion']
pagos_totales_liv3.fillna(0,inplace=True)
pagos_totales_liv3['Facturacion'] = pagos_totales_liv3['Facturacion'].round(2)
pagos_totales_liv3['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_totales_liv3['FechaFact'] = pd.to_datetime(pagos_totales_liv3['FechaFact'],dayfirst=True)
pagos_totales_liv3

,Division,Facturacion,FechaFact
0,Cof,202914.61,2021-01-25
1,Dictamen,2809.76,2021-01-25
2,Nacional2,208737.20,2021-01-25
3,Penacional,202896.40,2021-01-25
4,Suburbia,34437.01,2021-01-25


In [36]:
#Obtenemos datos de los gestores de Liverpool
promesas_mes_liv = pd.pivot_table(consolidado_pagos_liv,index=['segmento_y'],values=['monto_promesa'],aggfunc=['count',np.sum],dropna=False)
promesas_mes_liv = pd.DataFrame(promesas_mes_liv.to_records())
promesas_mes_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
promesas_mes_liv.fillna(0,inplace=True)
promesas_mes_liv

,Division,NumeroPromesas,SumaPromesas
0,Cof,94,468365.44
1,Dictamen,8,18735.00
2,Nacional2,129,664691.72
3,Penacional,178,866595.74
4,Suburbia,177,273933.82


In [37]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_liv = consolidado_pagos_liv.loc[(consolidado_pagos_liv['fecha_promesa']>=ayer)&(consolidado_pagos_liv['fecha_promesa']<=fin_mes)]
promesas_restantes_liv = pd.pivot_table(restantes_liv,index=['segmento_y'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_liv = pd.DataFrame(promesas_restantes_liv.to_records())
promesas_restantes_liv.columns = ['Division','PromesasRestantes']
promesas_mes_liv.fillna(0,inplace=True)
promesas_restantes_liv

,Division,PromesasRestantes
0,Cof,113978.00
1,Nacional2,101492.00
2,Penacional,95404.87
3,Suburbia,64694.10


In [38]:
pagos_totales_liv2

,Cliente,Facturacion,FechaFact
0,Liverpool,651794.98,2021-01-25


In [39]:
final_liv = pd.merge(pagos_totales_liv3,pagos_liv_gerencia,how='left',on='Division')
final_liv = pd.merge(final_liv,promesas_restantes_liv,how='left',on='Division')
#final_liv.insert(loc=0,column='Cliente',value='Liverpool')
final_liv.drop('SiguienteArancel',axis=1,inplace=True)
final_liv.insert(loc=0,column='Cliente',value='Liverpool')
final_liv

,Cliente,Division,Facturacion,FechaFact,Recuperado,RestSigArancel,PromesasRestantes
0,Liverpool,Cof,202914.61,2021-01-25,405829.22,194171,113978.00
1,Liverpool,Dictamen,2809.76,2021-01-25,17561.00,2.16994e+06,NaN
2,Liverpool,Nacional2,208737.20,2021-01-25,521842.99,1.10316e+06,101492.00
3,Liverpool,Penacional,202896.40,2021-01-25,724630.00,346799,95404.87
4,Liverpool,Suburbia,34437.01,2021-01-25,229580.06,1.77042e+06,64694.10


# Bradesco

In [40]:
#Hacemos la consulta referente a las gestiones de Bradesco
gestion_brad_mes = cs.query_naucalpan('bradescard_rpt_gestiones_detallado',mes,today)
gestion_brad_mes.columns = ['folio_gestion','firma_id','unegocio_id','NUMERO_TARJETA','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad_mes = gestion_brad_mes.loc[gestion_brad_mes['dictamen']=='PROMESA']
gestion_brad_mes['NUMERO_TARJETA'] = pd.to_numeric(gestion_brad_mes['NUMERO_TARJETA'])
gestion_brad_mes['monto_promesa'] = pd.to_numeric(gestion_brad_mes['monto_promesa'])
gestion_brad_mes = gestion_brad_mes.drop_duplicates(subset=['NUMERO_TARJETA','usuario','fecha_promesa','monto_promesa'])
gestion_brad_mes = gestion_brad_mes.loc[(gestion_brad_mes['fecha_promesa'].notnull())&(gestion_brad_mes['fecha_promesa']!='')]
gestion_brad_mes = gestion_brad_mes.loc[(gestion_brad_mes['fecha_promesa'].str[:4].map(int)<=2100)&(gestion_brad_mes['fecha_promesa'].str[:4].map(int)>=2000)]
gestion_brad_mes['fecha_promesa'] = pd.to_datetime(gestion_brad_mes['fecha_promesa'],dayfirst=False)

In [41]:
#Hacemos la union de asignacion y promesas
consolidado_brad_mes = pd.merge(gestion_brad_mes,asignacion_brad,how='left',on=['NUMERO_TARJETA'])
cols = ['monto_promesa']
consolidado_brad_mes[cols] = consolidado_brad_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']>50)]
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']<500000)]
#consolidado_brad_mes.fillna('COBRANZA',inplace=True)
consolidado_brad_mes['NUMERO_TARJETA'] = pd.to_numeric(consolidado_brad_mes['NUMERO_TARJETA'])

In [42]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [43]:
#Obtenemos el valor para cada segmento
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [44]:
#Funcion para asignar siguiente arancel
def sig_arancel(row):
    if row['Asignacion'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),c04pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),c04pv2,c04pv3))
    elif row['Asignacion'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),c05pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),c05pv2,c05pv3))
    elif row['Asignacion'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),c06pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),c06pv2,c06pv3))
    elif row['Asignacion'] == 'A':
        return np.where((A>0) & (A<=a1),a1,
                        np.where((A>a1) & (A<=a2),a2,a3))
    elif row['Asignacion'] == 'B':
        return np.where((B>0) & (B<=b1),b1,
                        np.where((B>b1) & (B<=b2),b2,b3))
    elif row['Asignacion'] == 'C1':
        return np.where((C1>0) & (C1<=c11),c11,
                        np.where((C1>c11) & (C1<=c12),c12,c13))
    elif row['Asignacion'] == 'C2':
        return np.where((C2>0) & (C2<=c21),c21,
                        np.where((C2>c21) & (C2<=c22),c22,c23))
    elif row['Asignacion'] == 'C3':
        return np.where((C3>0) & (C3<=c31),c31,
                        np.where((C3>c31) & (C3<=c32),c32,c33))
    elif row['Asignacion'] == 'C4':
        return np.where((C4>0) & (C4<=c41),c41,
                        np.where((C4>c41) & (C4<=c42),c42,c43))
    
    else:
        return 0

In [45]:
#Funcion para asignar arancel
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.04)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.04)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.04)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.04)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.04)))
    
    else:
        return 0

In [46]:
#Aplicamos funcion para asignar arancel
pagos_brad2 = pagos_brad
pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]
pagos_brad2['Cliente'] = 'Bradesco'

In [47]:
pagos_totales_brad2 = pd.pivot_table(pagos_brad2,index=['Cliente'],values=['Facturacion'],aggfunc=[np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2.columns = ['Cliente','Facturacion']
pagos_totales_brad2.fillna(0,inplace=True)
pagos_totales_brad2['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_totales_brad2

,Cliente,Facturacion,FechaFact
0,Bradesco,199040.1783,2021-01-21


In [48]:
pagos_brad_gerencia = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO','Facturacion'],aggfunc=[np.sum])
pagos_brad_gerencia = pd.DataFrame(pagos_brad_gerencia.to_records())
pagos_brad_gerencia.columns = ['Asignacion','Facturacion','Recuperado']
pagos_brad_gerencia.fillna(0,inplace=True)
pagos_brad_gerencia['SiguienteArancel'] = pagos_brad_gerencia.apply(sig_arancel, axis=1)
pagos_brad_gerencia['RestSigArancel'] = pagos_brad_gerencia['SiguienteArancel']-pagos_brad_gerencia['Recuperado']
cols = ['Facturacion','Recuperado','SiguienteArancel','RestSigArancel']
pagos_brad_gerencia[cols] = pagos_brad_gerencia[cols].round(2)
pagos_brad_gerencia = pagos_brad_gerencia.iloc[:,[0,1,2,4]]
pagos_brad_gerencia.columns = ['Division','Facturacion','Recuperado','RestSigArancel']
pagos_brad_gerencia

,Division,Facturacion,Recuperado,RestSigArancel
0,A,23473.26,86938.00,173252
1,B,6341.00,25364.00,189719
2,C0 4PV,33309.52,475850.32,728854
3,C0 5PV,17941.69,199352.07,462917
4,C0 6PV,15804.45,158044.50,649128
5,C1,39580.93,232829.00,1.19816e+06
6,C2,36974.54,194602.84,432031
7,C3,20586.30,98030.00,148395
8,C4,5028.49,21863.00,101666


In [49]:
#Obtenemos datos de las promesas de Bradesco
# prom_brad_div_mes = pd.pivot_table(consoliado_pagos_brad2,index='PAGOSVENCIDOS',values=['monto_promesa'], aggfunc = ['count',np.sum], margins=True,dropna=False)
# prom_brad_div_mes = pd.DataFrame(prom_brad_div_mes.to_records())
# prom_brad_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
# prom_brad_div_mes.fillna(0,inplace=True)
# prom_brad_div_mes

In [50]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_brad = consolidado_brad_mes.loc[(consolidado_brad_mes['fecha_promesa']>=ayer)&(consolidado_brad_mes['fecha_promesa']<=fin_mes)]
promesas_restantes_brad = pd.pivot_table(restantes_brad,index=['PAGOSVENCIDOS'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_brad = pd.DataFrame(promesas_restantes_brad.to_records())
promesas_restantes_brad.columns = ['Division','PromesasRestantes']
promesas_restantes_brad.fillna(0,inplace=True)
promesas_restantes_brad

,Division,PromesasRestantes
0,A,6868.0
1,B,5864.0
2,C0 4PV,212773.5
3,C0 5PV,146938.0
4,C0 6PV,42641.0
5,C1,128301.0
6,C2,91624.0
7,C3,27696.0
8,C4,10183.0


In [51]:
final_brad = pd.merge(pagos_brad_gerencia,promesas_restantes_brad,on='Division',how='left')
final_brad.insert(loc=0,column='Cliente',value='Bradesco')
final_brad['FechaFact'] = max(pagos_brad['FECHA_PAGO'])
final_brad

,Cliente,Division,Facturacion,Recuperado,RestSigArancel,PromesasRestantes,FechaFact
0,Bradesco,A,23473.26,86938.00,173252,6868.0,2021-01-21
1,Bradesco,B,6341.00,25364.00,189719,5864.0,2021-01-21
2,Bradesco,C0 4PV,33309.52,475850.32,728854,212773.5,2021-01-21
3,Bradesco,C0 5PV,17941.69,199352.07,462917,146938.0,2021-01-21
4,Bradesco,C0 6PV,15804.45,158044.50,649128,42641.0,2021-01-21
5,Bradesco,C1,39580.93,232829.00,1.19816e+06,128301.0,2021-01-21
6,Bradesco,C2,36974.54,194602.84,432031,91624.0,2021-01-21
7,Bradesco,C3,20586.30,98030.00,148395,27696.0,2021-01-21
8,Bradesco,C4,5028.49,21863.00,101666,10183.0,2021-01-21


# Credifiel

In [52]:
#Hacemos la consulta referente a las gestiones de Credifiel
gestion_cred = cs.query_naucalpan('credifiel_rpt_gestiones_detallado',mes,today)
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred['monto_promesa'] = pd.to_numeric(gestion_cred['monto_promesa'])
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
gestion_cred['credito'] = [x[3:] for x in gestion_cred['credito']]
gestion_cred['credito'] = pd.to_numeric(gestion_cred['credito'])
gestion_cred = gestion_cred.loc[(gestion_cred['fecha_promesa'].notnull())&(gestion_cred['fecha_promesa']!='')]
gestion_cred = gestion_cred.loc[(gestion_cred['fecha_promesa'].str[:4].map(int)<=2100)&(gestion_cred['fecha_promesa'].str[:4].map(int)>=2000)]
gestion_cred['fecha_promesa'] = pd.to_datetime(gestion_cred['fecha_promesa'],dayfirst=False)

In [53]:
##Hacemos el consolidado de la asignacion y promesado
consolidado_cred = pd.merge(gestion_cred,asignacion_cred,how='left',left_on=['credito'],right_on=['CREDITO'])
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred.loc[consolidado_cred['monto_promesa']>50]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]
consolidado_cred['fecha_promesa'] = pd.to_datetime(consolidado_cred['fecha_promesa'],dayfirst=False)
consolidado_cred.replace('CONTACTO','EXTRA',inplace=True)

In [54]:
pagos_cred3 = pd.merge(pagos_cred,asignacion_cred,how='left',left_on=['ID'],right_on=['CREDITO'])
pagos_cred3.dropna(subset=['ARANCEL'],inplace=True)
pagos_cred3['PagoFacturacion'] = [x*y for x,y in zip(pagos_cred3['ARANCEL'],pagos_cred3['IMPORTE'])]
pagos_cred3['division'] = 'EXTRA'
pagos_cred3 = pagos_cred3.drop_duplicates(subset=['ID','IMPORTE'])

In [55]:
#Obtenemos datos de las promesas de Bradesco
prom_cred_div_mes = pd.pivot_table(consolidado_cred,index=['dictamen'],values=['monto_promesa'], aggfunc=['count',np.sum], dropna=False)
prom_cred_div_mes = pd.DataFrame(prom_cred_div_mes.to_records())
prom_cred_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
prom_cred_div_mes

,Division,NumeroPromesas,SumaPromesas
0,EXTRA,121,331724.74


In [56]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_cred = consolidado_cred.loc[consolidado_cred['fecha_promesa']>=ayer]

promesas_restantes_cred = pd.pivot_table(restantes_cred,index=['dictamen'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_cred = pd.DataFrame(promesas_restantes_cred.to_records())
promesas_restantes_cred.columns = ['Division','PromesasRestantes']
promesas_restantes_cred.fillna(0,inplace=True)
promesas_restantes_cred

,Division,PromesasRestantes
0,EXTRA,136014.48


In [57]:
pagos_totales_cred = pd.pivot_table(pagos_cred3,index=['division'],values=['IMPORTE','PagoFacturacion'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
pagos_totales_cred = pagos_totales_cred.iloc[:,[0,1,3,4]]
pagos_totales_cred.columns = ['Division','NumeroPagos','SumaPagos','Facturacion']
pagos_totales_cred.fillna(0,inplace=True)
#pagos_totales_cred[['SumaPagos','SumaFacturacion']] = pagos_totales_cred[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
pagos_totales_cred['FechaFact'] = max(pagos_cred3['FECHA'])
pagos_totales_cred.insert(loc=0,column='Cliente',value='Credifiel')
pagos_totales_cred

,Cliente,Division,NumeroPagos,SumaPagos,Facturacion,FechaFact
0,Credifiel,EXTRA,65,294262.52,52775.833,2021-01-21


In [58]:
# pagos_cred3['Cliente'] = 'Credifiel'
# pagos_totales_cred2 = pd.pivot_table(pagos_cred3,index=['Cliente'],values=['IMPORTE','PagoFacturacion'],aggfunc=['count',np.sum],dropna=False)
# pagos_totales_cred2 = pd.DataFrame(pagos_totales_cred2.to_records())
# pagos_totales_cred2 = pagos_totales_cred2.iloc[:,[0,1,3,4]]
# pagos_totales_cred2.columns = ['Division','NumeroPagos','SumaPagos','Facturacion']
# pagos_totales_cred2.fillna(0,inplace=True)
# # pagos_totales_cred[['SumaPagos','SumaFacturacion']] = pagos_totales_cred[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
# pagos_totales_cred2['FechaFact'] = max(pagos_cred3['FECHA'])
# pagos_totales_cred2

In [59]:
final_cred = pd.merge(pagos_totales_cred,prom_cred_div_mes,how='left',on=['Division'])
final_cred = pd.merge(final_cred,promesas_restantes_cred,how='left',on=['Division'])
final_cred.fillna(0,inplace=True)
final_cred = final_cred.replace('EXTRA','Extrajudicial')
final_cred = final_cred.iloc[:,[0,1,4,5,8]]
final_cred

,Cliente,Division,Facturacion,FechaFact,PromesasRestantes
0,Credifiel,Extrajudicial,52775.833,2021-01-21,136014.48


# Banco Azteca

In [60]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
gestion_baz = cs.query_naucalpan('baz_rpt_gestiones_detallado',mes,today)
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
gestion_baz['monto_promesa'] = pd.to_numeric(gestion_baz['monto_promesa'])
gestion_baz = gestion_baz.loc[gestion_baz['fecha_promesa'].map(str).str[:4].map(int)<2100]
gestion_baz['fecha_promesa'] = pd.to_datetime(gestion_baz['fecha_promesa'],dayfirst=False)
#gestion_baz.head(3)

In [61]:
#Obtenemos datos de las promesas de Banco Azteca
gestion_baz = gestion_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
prom_baz_div_mes = pd.pivot_table(gestion_baz,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
prom_baz_div_mes = pd.DataFrame(prom_baz_div_mes.to_records())
prom_baz_div_mes = prom_baz_div_mes.iloc[:,[0,1,2]]
prom_baz_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
prom_baz_div_mes.fillna(0,inplace=True)
prom_baz_div_mes = prom_baz_div_mes.replace('PROMESA','COBRANZA')
prom_baz_div_mes

,Division,NumeroPromesas,SumaPromesas
0,COBRANZA,2090,1903120.0


In [62]:
#Obtenemos las promesas restantes a partir del día de ayer
restantes_baz = gestion_baz.loc[(gestion_baz['fecha_promesa']>=ayer)&(gestion_baz['fecha_promesa']<=fin_mes)]

promesas_restantes_baz = pd.pivot_table(restantes_baz,index=['dictamen'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_baz = pd.DataFrame(promesas_restantes_baz.to_records())
promesas_restantes_baz.columns = ['Division','PromesasRestantes']
promesas_restantes_baz = promesas_restantes_baz.replace('PROMESA','COBRANZA')
promesas_restantes_baz.fillna(0,inplace=True)
promesas_restantes_baz

,Division,PromesasRestantes
0,COBRANZA,304693.0


In [63]:
pagos_totales_baz_recovery =pd.pivot_table(fact_baz_rec,index=['Division'],values=['Total Recuperado','Total Comision'],aggfunc=[np.sum],dropna=False)
pagos_totales_baz_recovery = pd.DataFrame(pagos_totales_baz_recovery.to_records())
pagos_totales_baz_recovery.columns = ['Cliente','Recuperado','SumaPagos']
porcentaje_prom_arancel_rec = pagos_totales_baz_recovery['Recuperado']/pagos_totales_baz_recovery['SumaPagos']
porcentaje_prom_arancel_rec = porcentaje_prom_arancel_rec.item()
porcentaje_prom_arancel_rec

0.2354237040373463

In [64]:
pagos_totales_baz_26 = pd.pivot_table(fact_baz_26_55,index=['Division'],values=['Total Recuperado','Total Comision'],aggfunc=[np.sum],dropna=False)
pagos_totales_baz_26 = pd.DataFrame(pagos_totales_baz_26.to_records())
pagos_totales_baz_26.columns = ['Cliente','Recuperado','SumaPagos']
porcentaje_prom_arancel__26 = pagos_totales_baz_26['Recuperado']/pagos_totales_baz_26['SumaPagos']
porcentaje_prom_arancel__26 = porcentaje_prom_arancel__26.item()
porcentaje_prom_arancel__26

0.17015376908381225

In [65]:
pagos_baz_recovery['Recuperado'] = pagos_baz_recovery['RECUPERACION DE CAPITAL']+pagos_baz_recovery['RECUPERACION DE MORATORIOS']
pagos_baz_26['Recuperado'] = pagos_baz_26['RECUPERACION DE CAPITAL']+pagos_baz_26['RECUPERACION DE MORATORIOS']

In [66]:
pagos_baz_recovery['Facturacion'] = [x*porcentaje_prom_arancel_rec for x in pagos_baz_recovery['Recuperado']]
pagos_fact_baz_rec = pd.pivot_table(pagos_baz_recovery,index=['Division'],values=['Facturacion'],aggfunc=[np.sum])
pagos_fact_baz_rec = pd.DataFrame(pagos_fact_baz_rec.to_records())
pagos_fact_baz_rec.columns = ['Division_rec','Facturacion_rec']
pagos_fact_baz_rec['FechaFact'] = max(pagos_baz_recovery['DIA'])
pagos_fact_baz_rec.insert(loc=0,column='Cliente',value='Banco Azteca')
pagos_fact_baz_rec

,Cliente,Division_rec,Facturacion_rec,FechaFact
0,Banco Azteca,Recovery,214625.667752,2021-01-24


In [67]:
pagos_baz_26['Facturacion'] = [x*porcentaje_prom_arancel__26 for x in pagos_baz_26['Recuperado']]
# pagos_baz['Facturacion'] = pagos_baz['Arancel']*pagos_baz['Recuperado']
pagos_fact_baz_26 = pd.pivot_table(pagos_baz_26,index=['Division'],values=['Facturacion'],aggfunc=[np.sum])
pagos_fact_baz_26 = pd.DataFrame(pagos_fact_baz_26.to_records())
pagos_fact_baz_26.columns = ['Division_26','Facturacion_26']
pagos_fact_baz_26

,Division_26,Facturacion_26
0,26-39,174356.056719


In [68]:
pagos_fact_baz = pd.concat([pagos_fact_baz_rec,pagos_fact_baz_26],axis=1)
pagos_fact_baz['Division'] = 'Cobranza'
pagos_fact_baz['Facturacion'] = pagos_fact_baz['Facturacion_rec']+pagos_fact_baz['Facturacion_26']
pagos_fact_baz = pagos_fact_baz[['Cliente','Division','Facturacion','FechaFact']]
pagos_fact_baz

,Cliente,Division,Facturacion,FechaFact
0,Banco Azteca,Cobranza,388981.72447,2021-01-24


In [69]:
toadd = [pagos_fact_baz,promesas_restantes_baz]
final_baz = pd.concat(toadd,axis=1)
final_baz = final_baz.iloc[:,[0,1,2,3,5]]
final_baz

,Cliente,Division,Facturacion,FechaFact,PromesasRestantes
0,Banco Azteca,Cobranza,388981.72447,2021-01-24,304693.0


# TORREON

# Oriflame

In [70]:
reporte_pagos_oriflame = pd.pivot_table(pagos_oriflame,index=['Cliente'],values=['Factura'],aggfunc=np.sum)
reporte_pagos_oriflame = pd.DataFrame(reporte_pagos_oriflame.to_records())
reporte_pagos_oriflame.columns = ['Cliente','Facturacion']
reporte_pagos_oriflame['FechaFact'] = max(pagos_oriflame['FECHA_PAGO'])
reporte_pagos_oriflame['FechaFact'] = pd.to_datetime(reporte_pagos_oriflame['FechaFact'],dayfirst=True)
reporte_pagos_oriflame

,Cliente,Facturacion,FechaFact
0,Oriflame,17714.686,2021-01-11


# Arabela

In [71]:
reporte_pagos_arabela = pd.pivot_table(pagos_arabela,index=['Cliente'],values=['factura'],aggfunc=np.sum)
reporte_pagos_arabela = pd.DataFrame(reporte_pagos_arabela.to_records())
reporte_pagos_arabela.columns = ['Cliente','Facturacion']
reporte_pagos_arabela['FechaFact'] = max(pagos_arabela['FechaEntrega'])
reporte_pagos_arabela['FechaFact'] = pd.to_datetime(reporte_pagos_arabela['FechaFact'],dayfirst=True)
reporte_pagos_arabela

,Cliente,Facturacion,FechaFact
0,Arabela,30832.8702,2021-01-12


# Invex

In [72]:
reporte_pagos_invex = pd.pivot_table(pagos_invex,index=['Cliente'],values=['AMT_TRAN','HONORARIO'],aggfunc=[np.sum,np.mean])
reporte_pagos_invex = pd.DataFrame(reporte_pagos_invex.to_records())
reporte_pagos_invex = reporte_pagos_invex.iloc[:,[0,1,4]]
reporte_pagos_invex.columns = ['Cliente','Pago','Honorario']
reporte_pagos_invex['Facturacion'] = reporte_pagos_invex['Pago']*reporte_pagos_invex['Honorario']
reporte_pagos_invex = reporte_pagos_invex.iloc[:,[0,3]]
reporte_pagos_invex['FechaFact'] = max(pagos_invex['TRAN_POST_DT'])
reporte_pagos_invex['FechaFact'] = pd.to_datetime(reporte_pagos_invex['FechaFact'],dayfirst=True)
reporte_pagos_invex

,Cliente,Facturacion,FechaFact
0,Invex_REL,158078.144,1970-01-01 00:00:00.000044211
1,Invex_RLA,212830.868,1970-01-01 00:00:00.000044211
2,Invex_RLN,91735.102,1970-01-01 00:00:00.000044211


# Torreon

In [73]:
final_torreon = pd.read_excel(''+ruta+'Metas/Metas '+mes2+'.xlsx',sheet_name='Torreon')
final_torreon.insert(loc=0,column='Sucursal',value='Torreon')
final_torreon

,Sucursal,Cliente,Meta,Facturacion,FechaFact
0,Torreon,Avon,890000,171972.03,2021-01-20
1,Torreon,Oriflame,65000,38500.00,2021-01-26
2,Torreon,Invex,1095000,819614.00,2021-01-26
3,Torreon,Arabela,95000,65106.00,2021-01-26
4,Torreon,Cimaco,3500,547.00,2021-01-10
5,Torreon,Cristian Lay,6000,2896.00,2021-01-24


# Guatemala

In [74]:
final_guatemala = pd.read_excel(''+ruta+'Metas/Metas '+mes2+'.xlsx',sheet_name='Guatemala')
final_guatemala.insert(loc=0,column='Sucursal',value='Guatemala')
final_guatemala['Facturacion'] = final_guatemala['Facturacion']*quetzal
final_guatemala['Meta'] = final_guatemala['Meta']*quetzal
final_guatemala

,Sucursal,Cliente,Meta,Facturacion,FechaFact
0,Guatemala,Avon G,625000.0,222832.5,2021-01-21
1,Guatemala,Arabela G,12750.0,4235.0,2021-01-24
2,Guatemala,Banco Azteca,130000.0,35910.0,2021-01-24
3,Guatemala,Banco Credomatic,42500.0,19657.5,2021-01-22
4,Guatemala,Belcorp,12500.0,12467.5,2021-01-21
5,Guatemala,Banco Azteca,287500.0,131987.5,2021-01-27


# Panama

In [75]:
final_panama = pd.read_excel(''+ruta+'Metas/Metas '+mes2+'.xlsx',sheet_name='Panama')
final_panama.insert(loc=0,column='Sucursal',value='Panama')
final_panama['Facturacion'] = final_panama['Facturacion']*dolar
final_panama['Meta'] = final_panama['Meta']*dolar
final_panama

,Sucursal,Cliente,Meta,Facturacion,FechaFact
0,Panama,Cable Onda,56000,41440,2021-01-25
1,Panama,Jamar,160000,100000,2021-01-25
2,Panama,Banco Azteca,80000,117560,2021-01-24
3,Panama,Avon,40000,12000,2021-01-15
4,Panama,Adel Panama,60000,29780,2021-01-22
5,Panama,Cable Onda,40000,20160,2021-01-25
6,Panama,Rodelag,70000,0,2021-01-01
7,Panama,Banco Azteca,680000,821220,2021-01-24
8,Panama,Avon,25000,25000,2021-01-25


# Pagos sin promesa asignada

In [76]:
#Pagos de bradesco que no tienen promesa asignada
gestion_brad_mes2 = gestion_brad_mes.drop_duplicates(subset='NUMERO_TARJETA')
pagos_noasig_brad = pd.merge(pagos_brad,gestion_brad_mes2[['NUMERO_TARJETA','nombre_credito']],how='left',on='NUMERO_TARJETA')
pagos_noasig_brad = pagos_noasig_brad.loc[pagos_noasig_brad['nombre_credito'].isnull()]
pagos_noasig_brad_pivot = pd.pivot_table(pagos_noasig_brad,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'])
pagos_noasig_brad_pivot = pd.DataFrame(pagos_noasig_brad_pivot.to_records())
pagos_noasig_brad_pivot.columns = ['Division','RecuperadoSinPromesa','PagosNoPromesa']
pagos_noasig_brad_pivot

,Division,RecuperadoSinPromesa,PagosNoPromesa
0,A,49076.00,8
1,B,21817.00,2
2,C0 4PV,152128.32,123
3,C0 5PV,72912.04,64
4,C0 6PV,47234.50,38
5,C1,91747.00,64
6,C2,43231.84,34
7,C3,5899.00,8
8,C4,2897.00,2


In [77]:
#Pagos de Liverpool que no tienen promesa asignada
gestion_liv_mes2 = gestion_liv_mes.drop_duplicates(subset='credito')
pagos_noasig_liv = pd.merge(pagos_liv2,gestion_liv_mes2[['credito','nombre_credito']],left_on='Cuenta',right_on='credito',how='left')
pagos_noasig_liv = pagos_noasig_liv.loc[pagos_noasig_liv['nombre_credito'].isnull()]
pagos_noasig_liv_pivot = pd.pivot_table(pagos_noasig_liv,index=['segmento'],values=['Monto'],aggfunc=[np.sum,'count'])
pagos_noasig_liv_pivot = pd.DataFrame(pagos_noasig_liv_pivot.to_records())
pagos_noasig_liv_pivot.columns = ['Division','RecuperadoSinPromesa','PagosNoPromesa']
pagos_noasig_liv_pivot

,Division,RecuperadoSinPromesa,PagosNoPromesa
0,Cof,65361.78,16
1,Dictamen,4305.00,4
2,Nacional2,65047.99,37
3,Penacional,62322.00,24
4,Suburbia,30683.83,32


In [78]:
#Pagos de Liverpool que no tienen promesa asignada
gestion_cred2 = gestion_cred.drop_duplicates(subset='credito')
pagos_noasig_cred = pd.merge(pagos_cred,gestion_cred2[['credito','nombre_credito']],left_on='ID',right_on='credito',how='left')
pagos_noasig_cred = pagos_noasig_cred.loc[pagos_noasig_cred['nombre_credito'].isnull()]
pagos_noasig_cred['Cliente'] = 'Extrajudicial'
pagos_noasig_cred_pivot = pd.pivot_table(pagos_noasig_cred,index=['Cliente'],values=['IMPORTE'],aggfunc=[np.sum,'count'])
pagos_noasig_cred_pivot = pd.DataFrame(pagos_noasig_cred_pivot.to_records())
pagos_noasig_cred_pivot.columns = ['Division','RecuperadoSinPromesa','PagosNoPromesa']
pagos_noasig_cred_pivot

,Division,RecuperadoSinPromesa,PagosNoPromesa
0,Extrajudicial,206326.3,33


In [79]:
list1 = [pagos_noasig_cred_pivot,pagos_noasig_liv_pivot,pagos_noasig_brad_pivot]
pagos_noasig_naucalpan = pd.concat(list1,ignore_index=True)
pagos_noasig_naucalpan = pagos_noasig_naucalpan.iloc[:,:-1]
pagos_noasig_naucalpan

,Division,RecuperadoSinPromesa
0,Extrajudicial,206326.30
1,Cof,65361.78
2,Dictamen,4305.00
3,Nacional2,65047.99
4,Penacional,62322.00
5,Suburbia,30683.83
6,A,49076.00
7,B,21817.00
8,C0 4PV,152128.32
9,C0 5PV,72912.04


# Union Naucalpan

In [80]:
toadd_nau = [final_liv,final_brad,final_cred,final_baz]
final_naucalpan = pd.concat(toadd_nau,ignore_index=True)
final_naucalpan = pd.merge(final_naucalpan,pagos_noasig_naucalpan,on='Division',how='left')
final_naucalpan.insert(loc=0,column='Sucursal',value='Naucalpan')
final_naucalpan.insert(loc=0,column='Pais',value='Mexico')
final_naucalpan['TendenciaCierre1'] = [(30/x.day)*y for x,y in zip(final_naucalpan['FechaFact'],final_naucalpan['Facturacion'])]
final_naucalpan.fillna(0,inplace=True)
final_naucalpan

,Pais,Sucursal,Cliente,Division,Facturacion,FechaFact,Recuperado,RestSigArancel,PromesasRestantes,RecuperadoSinPromesa,TendenciaCierre1
0,Mexico,Naucalpan,Liverpool,Cof,202914.61000,2021-01-25,405829.22,194170.78,113978.00,65361.78,243497.532000
1,Mexico,Naucalpan,Liverpool,Dictamen,2809.76000,2021-01-25,17561.00,2169939.00,0.00,4305.00,3371.712000
2,Mexico,Naucalpan,Liverpool,Nacional2,208737.20000,2021-01-25,521842.99,1103157.01,101492.00,65047.99,250484.640000
3,Mexico,Naucalpan,Liverpool,Penacional,202896.40000,2021-01-25,724630.00,346798.60,95404.87,62322.00,243475.680000
4,Mexico,Naucalpan,Liverpool,Suburbia,34437.01000,2021-01-25,229580.06,1770419.94,64694.10,30683.83,41324.412000
5,Mexico,Naucalpan,Bradesco,A,23473.26000,2021-01-21,86938.00,173252.00,6868.00,49076.00,33533.228571
6,Mexico,Naucalpan,Bradesco,B,6341.00000,2021-01-21,25364.00,189719.00,5864.00,21817.00,9058.571429
7,Mexico,Naucalpan,Bradesco,C0 4PV,33309.52000,2021-01-21,475850.32,728853.68,212773.50,152128.32,47585.028571
8,Mexico,Naucalpan,Bradesco,C0 5PV,17941.69000,2021-01-21,199352.07,462916.93,146938.00,72912.04,25630.985714
9,Mexico,Naucalpan,Bradesco,C0 6PV,15804.45000,2021-01-21,158044.50,649127.50,42641.00,47234.50,22577.785714


In [81]:
pdList = [pagos_totales_liv2,pagos_totales_brad2,pagos_totales_cred,pagos_fact_baz]
final_naucalpan2 = pd.concat(pdList,ignore_index=True)
final_naucalpan2 = final_naucalpan2.iloc[:,:-3]
final_naucalpan2 = pd.merge(final_naucalpan2,meta_naucalpan,on=['Cliente'],how='left')
final_naucalpan2.insert(loc=0,column='Sucursal',value='Naucalpan')
final_naucalpan2
baz_renta = pd.DataFrame([['Naucalpan','Banco Azteca',meta_baz_renta*day/30,ayer,meta_baz_renta]],columns=['Sucursal','Cliente','Facturacion','FechaFact','Meta'])
final_naucalpan2 = pd.concat([final_naucalpan2,baz_renta],ignore_index=True)
final_naucalpan2

,Sucursal,Cliente,Facturacion,FechaFact,Meta
0,Naucalpan,Liverpool,651794.980000,2021-01-25,1150000
1,Naucalpan,Bradesco,199040.178300,2021-01-21,750000
2,Naucalpan,Credifiel,52775.833000,2021-01-21,100000
3,Naucalpan,Banco Azteca,388981.724470,2021-01-24,880000
4,Naucalpan,Banco Azteca,870133.333333,2021-01-27,1004000


In [82]:
toadd_final = [final_naucalpan2,final_torreon,final_guatemala,final_panama]
final_sucursales = pd.concat(toadd_final,ignore_index=True)

final_sucursales2 = pd.pivot_table(final_sucursales,index=['Sucursal','Cliente'],values=['Facturacion','Meta','FechaFact'],aggfunc=[np.sum,np.max])
final_sucursales2 = pd.DataFrame(final_sucursales2.to_records())
final_sucursales2 = final_sucursales2.iloc[:,[0,1,2,5,3]]
final_sucursales2.columns = ['Sucursal','Cliente','Facturacion','FechaFact','Meta']

final_sucursales2['Avance%'] = final_sucursales2['Facturacion']/final_sucursales2['Meta']
final_sucursales2['MetaDia'] = ((final_sucursales2['FechaFact'].dt.day/30)*final_sucursales2['Meta']).round()
final_sucursales2['TendenciaCierre'] = [(30/x.day)*y for x,y in zip(final_sucursales2['FechaFact'],final_sucursales2['Facturacion'])]
final_sucursales2['Semaforo'] = np.where(final_sucursales2['MetaDia']<=final_sucursales2['Facturacion'],'verde',
                                np.where((final_sucursales2['MetaDia']*.75<=final_sucursales2['Facturacion']) &                                       (final_sucursales2['MetaDia']>final_sucursales2['Facturacion']),'amarillo','rojo'))
fecha_fact = datetime.strftime(max(final_sucursales2['FechaFact']),'%d-%m-%Y')

final_sucursales2

,Sucursal,Cliente,Facturacion,FechaFact,Meta,Avance%,MetaDia,TendenciaCierre,Semaforo
0,Guatemala,Arabela G,4.235000e+03,2021-01-24,12750.0,0.332157,10200.0,5.293750e+03,rojo
1,Guatemala,Avon G,2.228325e+05,2021-01-21,625000.0,0.356532,437500.0,3.183321e+05,rojo
2,Guatemala,Banco Azteca,1.678975e+05,2021-01-27,417500.0,0.402150,375750.0,1.865528e+05,rojo
3,Guatemala,Banco Credomatic,1.965750e+04,2021-01-22,42500.0,0.462529,31167.0,2.680568e+04,rojo
4,Guatemala,Belcorp,1.246750e+04,2021-01-21,12500.0,0.997400,8750.0,1.781071e+04,verde
5,Naucalpan,Banco Azteca,1.259115e+06,2021-01-27,1884000.0,0.668320,1695600.0,1.399017e+06,rojo
6,Naucalpan,Bradesco,1.990402e+05,2021-01-21,750000.0,0.265387,525000.0,2.843431e+05,rojo
7,Naucalpan,Credifiel,5.277583e+04,2021-01-21,100000.0,0.527758,70000.0,7.539405e+04,amarillo
8,Naucalpan,Liverpool,6.517950e+05,2021-01-25,1150000.0,0.566778,958333.0,7.821540e+05,rojo
9,Panama,Adel Panama,2.978000e+04,2021-01-22,60000.0,0.496333,44000.0,4.060909e+04,rojo


In [83]:
writer = pd.ExcelWriter(''+ruta+'/'+mes2+'/Reportes diarios/Avance pagos/Avance Pagos '+fecha_fact+'.xlsx',engine='xlsxwriter')
final_sucursales2.to_excel(writer,'FactCliente',index=False,header=True)
final_naucalpan.to_excel(writer,'Naucalpan',index=False,header=True)

writer.save()